### Start and Run All

In [2]:
import os
import pandas as pd
from urllib import request
from bs4 import BeautifulSoup
# from datetime import date, timedelta
from datetime import datetime
from time import sleep, strftime
current_time = datetime.now()
current_time

datetime.datetime(2025, 1, 23, 21, 19, 9, 165856)

In [4]:
url = "https://www.set.or.th/en/market/product/stock/quote/"
percent_span = '' 

In [6]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [8]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [10]:
file_name = "name-ttl.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file : {input_file}") 

Input file : C:\Users\PC1\OneDrive\A5\Data\name-ttl.csv


In [12]:
file_name = "price-hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [14]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}")

Output file : C:\Users\PC1\OneDrive\A5\Data\price-hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\price-hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\price-hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\price-hilo.csv


In [16]:
def cvt_str_to_flt(input_str):
    # Check for the special pattern and return 0
    if input_str == '(-)':
        return 0
    
    # Remove parentheses and percent sign
    cleaned_str = input_str.replace('(', '').replace(')', '').replace('%', '')

    # Convert to float
    try:
        return float(cleaned_str)
    except ValueError:
        # Handle the case where the conversion fails
        return None
 

In [18]:
def get_stock_data(in_df):

    in_df.columns = ['name']
    
    output_columns = ['name', 'price', 'today_low', 'today_high', 'year_low', 'year_high', 'percent', 'updated_at']
    # Initialize an empty DataFrame with specified columns
    out_df = pd.DataFrame(columns=output_columns)
    
    for index, row in in_df.iterrows():
        name = str(row['name']).upper()
        response = request.urlopen(url + name + '/price')
        html_page = BeautifulSoup(response, 'html.parser')
        
  #      price = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info').text.strip()        
  #      if (price == '-'):
  #          price = '0.00'        

        price_div = html_page.find('div', class_='value text-white mb-0 me-2 lh-1 stock-info')
        if price_div:
            price = price_div.text.strip()
        else:
            print(f"Price div not found for {name}")
            price = '0.00'     
            
        # Try to find the <h3> tag with the class 'theme-success'
        h3_tag = html_page.find('h3', class_='theme-success')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-danger')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)    
                
        # Try to find the <h3> tag with the class 'theme-danger'
        h3_tag = html_page.find('h3', class_='theme-normal')
        if h3_tag:
        # Find all <span> tags within the found <h3> tag
            span_tags = h3_tag.find_all('span')
    
            if len(span_tags) > 1:
                # Extract the text from the second <span> tag
                percent_span = span_tags[1].text
                # print(percent_span)   
                
        percent = cvt_str_to_flt(percent_span)
                
        values_low = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder me-auto lh-1')
        today_low = values_low[0].text
        if (today_low == '-'):
            today_low = '0.00'
        
        values_high = html_page.find_all('span', class_='title-font-family fs-16px fw-bolder lh-1')
        today_high = values_high[0].text
        if (today_high == '-'):
            today_high = '0.00'         
        
        year_low = values_low[1].text        
        year_high = values_high[1].text     
        
        # Find the div with the specific class, then find the span within it
        div = html_page.find('div', class_='price-detail-date d-flex align-items-center text-nowrap fs-12px')
        if div:
            span = div.find('span')
            if span:
                text = span.get_text()
#                 print(text)
            else:
                print("Span not found")
        else:
            print("Div not found")
            
        # Extract the date part from the string
        # Assuming the date is always in the format 'dd Mon yyyy'
        date_str = ' '.join(text.split()[2:5])

        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, '%d %b %Y')

        # Format the datetime object to 'yyyy-mm-dd'
        updated_at = date_obj.strftime('%Y-%m-%d')

#         print(updated_at)
        
        # Append row to the DataFrame and print the row
        out_df.loc[len(out_df)] = [name, price, today_low, today_high, year_low, year_high, percent, updated_at]
#       print(out_df.iloc[-1].to_string(header=False, index=False)) 
        print(','.join(map(str, out_df.iloc[-1])))
        sleep(1)
        
    return out_df  # return the final dataframe with all values populated

In [20]:
start_time = strftime("%I:%M %p")
print('Start at: ', start_time)

Start at:  09:22 PM


In [22]:
df = pd.read_csv(input_file, header=None)
out_df = get_stock_data(df)

ACE,1.26,1.25,1.27,1.25,1.60,0.0,2025-01-23
ADVANC,283.00,283.00,288.00,193.00,299.00,-2.08,2025-01-23
AEONTS,113.50,113.50,118.50,107.00,168.50,-3.81,2025-01-23
AH,14.10,13.90,14.40,14.00,29.50,-0.7,2025-01-23
AIE,0.78,0.77,0.79,0.74,1.51,-1.27,2025-01-23
AIMIRT,10.40,10.40,10.50,10.20,11.20,-0.95,2025-01-23
AIT,4.26,4.24,4.38,3.62,5.05,-1.84,2025-01-23
AJ,2.48,2.44,2.56,2.36,7.15,1.64,2025-01-23
AMATA,26.75,26.75,27.50,20.10,31.00,-2.73,2025-01-23
ANAN,0.50,0.50,0.52,0.48,1.03,-1.96,2025-01-23
AOT,57.00,57.00,58.00,55.00,67.50,-2.15,2025-01-23
AP,7.75,7.75,7.85,7.50,11.20,-0.64,2025-01-23
ASIAN,7.75,7.75,7.85,6.35,11.20,0.0,2025-01-23
ASK,8.70,8.60,8.95,8.40,20.30,-1.69,2025-01-23
ASP,2.32,2.32,2.36,2.12,2.88,-0.85,2025-01-23
ASW,7.85,7.75,7.85,7.35,8.45,0.0,2025-01-23
AWC,3.08,3.06,3.12,2.88,4.58,-0.65,2025-01-23
BA,19.00,18.90,19.40,14.20,26.00,-2.56,2025-01-23
BAM,5.65,5.65,5.80,5.35,10.40,-0.88,2025-01-23
BANPU,5.15,5.15,5.35,4.38,7.50,-0.96,2025-01-23
BAY,23.80,23.80,24.00,22.20

In [24]:
end_time = strftime("%I:%M %p")
print('End at: ', end_time)

End at:  09:28 PM


In [26]:
out_df.query('price == "-"')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at


In [28]:
out_df.shape

(221, 8)

In [30]:
out_df = out_df[out_df['name'] != 'BCT']

In [36]:
out_df = out_df[out_df['name'] != 'LPF']

In [38]:
out_df = out_df[out_df['name'] != 'TYCN']

In [40]:
out_df = out_df[out_df['name'] != 'GRAMMY']

In [42]:
out_df = out_df[out_df['name'] != 'KYE']

In [28]:
out_df.shape

(221, 8)

In [30]:
out_df['price'] = out_df['price'].astype('float')
out_df['today_low'] = out_df['today_low'].astype('float')
out_df['today_high'] = out_df['today_high'].astype('float')
out_df['year_low'] = out_df['year_low'].astype('float')
out_df['year_high'] = out_df['year_high'].astype('float')
out_df['percent'] = out_df['percent'].astype('float')

In [32]:
sorted_out_df = out_df.sort_values('percent')

In [34]:
sorted_out_df.query('today_low < year_low')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
208,TU,11.30,11.20,12.00,11.80,16.00,-6.61,2025-01-23
157,SCGP,16.40,16.40,17.00,16.60,35.25,-5.20,2025-01-23
72,GLOBAL,11.80,11.70,12.40,12.40,17.40,-4.84,2025-01-23
112,M,18.30,18.20,18.90,18.90,39.50,-3.17,2025-01-23
155,SCC,150.00,150.00,155.00,150.50,278.00,-2.91,2025-01-23
220,WORK,5.25,5.20,5.40,5.25,11.10,-2.78,2025-01-23
124,OR,11.50,11.40,11.80,11.50,19.00,-2.54,2025-01-23
122,NOBLE,2.36,2.32,2.42,2.42,4.10,-2.48,2025-01-23
66,FPT,9.00,9.00,9.20,9.10,15.40,-2.17,2025-01-23
165,SKR,8.65,8.55,8.75,8.65,10.70,-1.14,2025-01-23


In [36]:
out_df.query('today_low < year_low').shape[0]

12

In [38]:
sorted_out_df.query('today_high > year_high')

,name,price,today_low,today_high,year_low,year_high,percent,updated_at
101,KTB,22.7,22.7,23.1,15.4,23.0,-1.3,2025-01-23


In [40]:
out_df.query('today_high > year_high').shape[0]

1

In [42]:
file_name = "Price-Hilo.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"output_file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

output_file : C:\Users\PC1\OneDrive\A5\Data\Price-Hilo.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\Price-Hilo.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\Price-Hilo.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\Price-Hilo.csv


In [46]:
out_df.to_csv(output_file, index=False)  
out_df.to_csv(god_file, index=False) 
out_df.to_csv(icd_file, index=False)  
out_df.to_csv(osd_file, index=False) 

In [48]:
current_time = datetime.now()
current_time

datetime.datetime(2025, 1, 23, 21, 31, 33, 177051)